# Bayesian pooled polling charts (Using Stan)

## Set-up

In [1]:
# system imports
# import itertools
from typing import Any
from pathlib import Path
import json

# analytic imports
# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
import cmdstanpy
from cmdstanpy import CmdStanModel

In [2]:
# local import
# import bayes_tools
# import plotting
from common import VOTING_INTENTION, ensure  # MIDDLE_DATE
from data_capture import retrieve

### Install CmdStan

In [3]:
def install_cmdstan() -> Path:
    """Install CmdStan (if needed) and
    returns a stan working directory Path.

    Note: Takes up to a couple of minutes
    if CmdStan is not installed."""

    # Install Stan
    print(f"cmdstanpy version: {cmdstanpy.__version__}")
    cmdstanpy.install_cmdstan()

    # Create a stan working directory (if needed)
    directory = Path("./stan-work/")
    directory.mkdir(parents=True, exist_ok=True)

    return directory


STAN_DIR = install_cmdstan()

cmdstanpy version: 1.2.5
CmdStan install directory: /Users/bryanpalmer/.cmdstan
CmdStan version 2.36.0 already installed
Test model compilation


In [4]:
# plotting related
MODEL_DIR = "../model-images/"
Path(MODEL_DIR).mkdir(parents=True, exist_ok=True)

SHOW = True  # show charts in the notebook
SHOW_MODEL_MAPS = False  # show model maps in notebook

## Get the raw input data

In [5]:
data = retrieve()
ensure(bool(data), "You must run the data capture notebook every day.")
print(f"Latest poll (mean date): {data[VOTING_INTENTION].iloc[:, -2].max()}")
print(f"Data columns:\n{data[VOTING_INTENTION].columns}")
print(
    f"Poll count by pollster:\n{data[VOTING_INTENTION]['Brand'].value_counts().sort_index()}"
)

Latest poll (mean date): 2025-01-12
Data columns:
Index(['Date', 'Brand', 'Interview mode', 'Sample size', 'Primary vote L/NP',
       'Primary vote ALP', 'Primary vote GRN', 'Primary vote ONP',
       'Primary vote UAP', 'Primary vote OTH', '2pp vote ALP', '2pp vote L/NP',
       'First Date', 'Mean Date', 'Last Date'],
      dtype='object')
Poll count by pollster:
Brand
ANU                     1
DemosAU                 1
Dynata                  1
Essential              21
Essential 2            28
Freshwater Strategy    16
Newspoll               30
Newspoll-YouGov         3
RedBridge Group         9
Resolve Strategic      17
Resolve Strategic 2    11
Roy Morgan             66
Wolf & Smith            1
YouGov                 16
Name: count, dtype: int64


## Do the Bayesian pooling ...

### Specify the aggregation model

In [6]:
# The model
MODEL_TEXT = """
data {
    // data sizes
    int<lower=1> n_polls;   // number of polls
    int<lower=1> n_days;    // number of days from first to last poll
    int<lower=1> n_houses;  // number of pollsters

    // other initialisation data
    real<lower=-3,upper=3> y_start;       // initial voting intention guess
    real<lower=0,upper=2> polling_error;  // typical polling error (after scaling)
    real<lower=0,upper=0.5> daily_drift;  // typical daily drift in scaled voting intention

    // polling data
    array[n_polls] real<lower=-5, upper=5> y;          // adj poll estimates
    array[n_polls] int<lower=1,upper=n_houses> house;  // pollster number
    array[n_polls] int<lower=1,upper=n_days> day;      // polling day
}
parameters {
    array[n_days] real<lower=-5,upper=5> hidden_voting_intention;
    vector[n_houses-1] rawHouseEffects;
    real<lower=0,upper=0.5> daily_sigma;
}
transformed parameters {
    // sum to zero constraint for house effects
    vector[n_houses] houseEffect ;
    houseEffect = append_row(rawHouseEffects, -sum(rawHouseEffects));
}
model{
    // -- house effects model: pollsters have inate bias
    //    that collectively sums to zero
    rawHouseEffects ~ normal(0, 1.0); // weakly informative

    // -- temporal model: voting intention drifts slowly each day
    hidden_voting_intention[1] ~ normal(y_start, 0.5);
    for(i in 2:n_days) {
        hidden_voting_intention[i] 
        ~ normal(hidden_voting_intention[i-1], daily_drift);
    }

    // -- observational model, polling data is noisy and biased
    //    around the true/hidden voting intention
    for(poll in 1:n_polls) {
        y[poll] ~ normal(
            houseEffect[house[poll]] + hidden_voting_intention[day[poll]], 
            polling_error
        );
    }
}
"""

In [7]:
def compile_stan_model(text: str) -> CmdStanModel:
    """Compile and return the Stan model."""

    # write the model to a .stan file
    filepath = STAN_DIR / "model.stan"
    with filepath.open("w") as f:
        f.write(text)

    # compile the model
    model = CmdStanModel(stan_file=filepath)

    # print compiled model info
    print(model)
    print(model.exe_info())
    print("Model compiled successfully.")

    return model


MODEL = compile_stan_model(MODEL_TEXT)

08:31:52 - cmdstanpy - INFO - compiling stan file /Users/bryanpalmer/Australian-Federal-Election-2025/notebooks/stan-work/model.stan to exe file /Users/bryanpalmer/Australian-Federal-Election-2025/notebooks/stan-work/model
08:31:56 - cmdstanpy - INFO - compiled model executable: /Users/bryanpalmer/Australian-Federal-Election-2025/notebooks/stan-work/model


CmdStanModel: name=model
	 stan_file=/Users/bryanpalmer/Australian-Federal-Election-2025/notebooks/stan-work/model.stan
	 exe_file=/Users/bryanpalmer/Australian-Federal-Election-2025/notebooks/stan-work/model
	 compiler_options=stanc_options={}, cpp_options={}
{}
Model compiled successfully.


### Fit the data to the model

In [8]:
def prepare_input_data(
    field="2pp vote ALP",
) -> tuple[Path, dict[str, Any], dict[str, Any]]:
    """Compile the data for the model.
    Build the model inputs and context.
    Save the model inputs to a file.
    Return a tuple comprising:
    - the path to the model inputs file,
    - the model inputs, and
    - the model context dictionary."""

    def report(d: dict[str, Any]) -> None:
        for k, v in d.items():
            print(f"{k}: {v}")
        print("-------------\n")

    vote = data[VOTING_INTENTION].loc[data[VOTING_INTENTION][field].notnull()].copy()
    mean_offset = vote[field].mean()
    scale = 5  # pick a scale to make the data within the range -2 to 2
    y = (vote[field] - mean_offset) / scale  # scale to -2 to 2
    house = vote["Brand"].astype("category").cat.codes
    house_codes = dict(zip(house, vote["Brand"]))
    n_houses = len(house_codes)
    first_date = vote["Mean Date"].min()
    day = (vote["Mean Date"] - first_date).apply(lambda x: x.n)

    model_inputs = {
        #-- scalar values - model parameters
        "n_polls": len(y),  # number of polls
        "n_days": int(day.max() + 1),  # inclusive number of days from first to last poll
        "n_houses": n_houses,
        "polling_error": 2 / scale,  # weakly informative prior
        "daily_drift": 1 / scale,  # weakly informative prior 
        "y_start": y[:10].mean(),  # initial guess for the hidden variable

        # -- arrays of polling data
        "y": y.tolist(),
        "house": (house + 1).tolist(),  # Stan is 1-indexed
        "day": (day + 1).tolist(),  # Stan is 1-indexed
    }
    datapath = STAN_DIR / f"data-{field.replace(' ', '-').replace('/', '')}.json"
    with datapath.open("w") as f:
        json.dump(model_inputs, f)
    print(f"Model inputs saved to: {datapath}")
    report(model_inputs)

    model_context = {
        "field": field,
        "first day": first_date,
        "mean offset": mean_offset,
        "scale": scale,
        "house codes": house_codes,
    }
    print("Context:")
    report(model_context)

    return datapath, model_inputs, model_context

In [9]:
def aggregate_series(field):
    """Produce a Bayesian aggregation of the voting intention
    for a given field (e.g. '2pp vote ALP')."""

    datapath,_model_inputs, _model_context = prepare_input_data(field)

    fit = MODEL.sample(
        data=datapath,
        show_console=True,
        iter_sampling=1000,
        iter_warmup=500,
        chains=4,
    )

    print(fit.summary())

In [10]:
def for_all_fields(fields: list[str] | None = None):
    """Undertake Bayesian aggregation for a selection of fields."""

    if fields is None:
        fields = [
            '2pp vote L/NP', '2pp vote ALP',
            'Primary vote L/NP', 'Primary vote ALP', 
            'Primary vote GRN', 'Primary vote OTH'
        ]   

    for field in fields:
        print(f"Field: {field}")
        aggregate_series(field)


for_all_fields()

08:31:57 - cmdstanpy - INFO - Chain [1] start processing
08:31:57 - cmdstanpy - INFO - Chain [2] start processing
08:31:57 - cmdstanpy - INFO - Chain [3] start processing
08:31:57 - cmdstanpy - INFO - Chain [4] start processing
08:31:57 - cmdstanpy - INFO - Chain [1] done processing
08:31:57 - cmdstanpy - INFO - Chain [2] done processing
08:31:57 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6
08:31:57 - cmdstanpy - ERROR - Chain [2] error: terminated by signal 6 Unknown error: -6
08:31:57 - cmdstanpy - INFO - Chain [3] done processing
08:31:57 - cmdstanpy - ERROR - Chain [3] error: terminated by signal 6 Unknown error: -6
08:31:57 - cmdstanpy - INFO - Chain [4] done processing
08:31:57 - cmdstanpy - ERROR - Chain [4] error: terminated by signal 6 Unknown error: -6


Field: 2pp vote L/NP
Model inputs saved to: stan-work/data-2pp-vote-LNP.json
n_polls: 221
n_days: 943
n_houses: 14
polling_error: 0.4
daily_drift: 0.2
y_start: -0.8319779815971913
y: [-0.042871598618468454, -0.20287159861846787, -0.8028715986184679, -1.8028715986184678, -1.002871598618468, -0.9028715986184679, -1.3028715986184678, -1.202871598618468, -0.6028715986184678, -0.4539354284057012, -0.6028715986184678, -1.6028715986184678, -0.3397137038816254, -0.4028715986184679, -1.6028715986184678, -0.7607663354605734, -1.002871598618468, -0.6028715986184678, -1.1818189670395198, -1.3028715986184678, -1.202871598618468, -0.5706135341023397, -0.9028715986184679, -0.4028715986184679, -0.5028715986184679, -0.1405060072206183, -1.6028715986184678, -0.5502400196710994, -0.6028715986184678, -0.7607663354605734, -0.5502400196710994, -1.9028715986184679, -0.8028715986184679, -0.8794673432993193, -1.8028715986184678, -0.6028715986184678, -0.7607663354605734, -0.0028715986184678854, -0.5502400196710

RuntimeError: Error during sampling:

Command and output files:
RunSet: chains=4, chain_ids=[1, 2, 3, 4], num_processes=4
 cmd (chain 1):
	['/Users/bryanpalmer/Australian-Federal-Election-2025/notebooks/stan-work/model', 'id=1', 'random', 'seed=45661', 'data', 'file=stan-work/data-2pp-vote-LNP.json', 'output', 'file=/var/folders/96/8bhlz_x975z93glbxq_2_yqr0000gn/T/tmpd17ivflf/modelv65vlmk3/model-20250120083157_1.csv', 'method=sample', 'num_samples=1000', 'num_warmup=500', 'algorithm=hmc', 'adapt', 'engaged=1']
 retcodes=[-6, -6, -6, -6]
 per-chain output files (showing chain 1 only):
 csv_file:
	/var/folders/96/8bhlz_x975z93glbxq_2_yqr0000gn/T/tmpd17ivflf/modelv65vlmk3/model-20250120083157_1.csv
 console_msgs (if any):
	/var/folders/96/8bhlz_x975z93glbxq_2_yqr0000gn/T/tmpd17ivflf/modelv65vlmk3/model-20250120083157_0-stdout.txt

## Still to do

In [ ]:
# to do

## Finished

In [12]:
%load_ext watermark
%watermark --python --machine --conda --iversions --watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.12.8
IPython version      : 8.31.0

conda environment: stan

Compiler    : Clang 18.1.8 
OS          : Darwin
Release     : 24.2.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

json     : 2.0.9
cmdstanpy: 1.2.5

Watermark: 2.5.0



In [ ]:
print("Finished")